In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.1 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer, util

In [3]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
news = pd.read_table('news.tsv', header = None)

title = news.columns == 3

body = news.columns == 4

news = news[news.columns[[0,3,4]]]
news = news.drop_duplicates(subset=None, keep='first', inplace=False)
news = news.rename(columns={0: 'ID', 3: 'Title', 4: 'Description'})
news_cleaned = news.dropna()
news_cleaned['Text'] = news_cleaned['Title'] + '[SEP]' + news_cleaned['Description']
del news

<ipython-input-5-6e6cf6e730df>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_cleaned['Text'] = news_cleaned['Title'] + '[SEP]' + news_cleaned['Description']


In [ ]:
news_cleaned[news_cleaned['ID'] == 'N20809']

,ID,Title,Description,Text
1096,N20809,7 Reasons Your Skin Has Red Spots and Bumps,Dermatologists explain what these red marks ar...,7 Reasons Your Skin Has Red Spots and Bumps[SE...


In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np
corpus_embeddings = model.encode(news_cleaned['Text'].tolist(), convert_to_tensor=True, normalize_embeddings=True)

#corpus_embeddings_df = pd.DataFrame(corpus_embeddings)
corpus_embeddings

tensor([[ 0.0334, -0.0090,  0.0018,  ..., -0.0056, -0.0548, -0.0152],
        [ 0.0384,  0.0382,  0.0482,  ...,  0.0052,  0.0087,  0.0030],
        [-0.0017,  0.0299,  0.0176,  ...,  0.0021, -0.0065, -0.0035],
        ...,
        [ 0.0246,  0.0586,  0.0465,  ...,  0.0349, -0.0147,  0.0415],
        [-0.0163,  0.0343, -0.0015,  ...,  0.0107, -0.0600,  0.0100],
        [-0.0338,  0.0514,  0.0014,  ..., -0.0433,  0.0333, -0.0097]],
       device='cuda:0')

In [ ]:
query = news_cleaned.loc[4, "Text"]

query_embedding = model.encode(query,convert_to_tensor=True, normalize_embeddings=True)

In [ ]:
top_k=5
similar_indices = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)[0]
similar_indices

[{'corpus_id': 4, 'score': 0.9999999403953552},
 {'corpus_id': 8558, 'score': 0.8248497247695923},
 {'corpus_id': 868, 'score': 0.6882694363594055},
 {'corpus_id': 1049, 'score': 0.6168100237846375},
 {'corpus_id': 29814, 'score': 0.6159457564353943}]

In [ ]:
similar_ids = [news_cleaned['ID'][idx['corpus_id']] for idx in similar_indices]

recommendations = news_cleaned[news_cleaned['ID'].isin(similar_ids)]
recommendations

,ID,Title,Description,Text
4,N38324,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...","How to Get Rid of Skin Tags, According to a De..."
868,N14017,UFC on ESPN 6 matchmaker: Who's next for Chris...,MMA junkie's Mike Bohn plays matchmaker and lo...,UFC on ESPN 6 matchmaker: Who's next for Chris...
1049,N25813,Watch 4 Porsches Lap a Tesla Model S Broken Do...,The Tesla-Porsche rivalry heats up a few more ...,Watch 4 Porsches Lap a Tesla Model S Broken Do...
8558,N40291,"'Resiliency, Strength and Community Collaborat...",Sunday is the 1-year anniversary of the deadli...,"'Resiliency, Strength and Community Collaborat..."
29814,N60149,New hiking trail to open east of the Anacostia...,The Pope Branch Trail is the first unpaved tra...,New hiking trail to open east of the Anacostia...


NameError: name 'cosine_similarity' is not defined

In [ ]:
cosine_similarities = util.pytorch_cos_sim(corpus_embeddings, corpus_embeddings)
cosine_similarities

tensor([[ 1.0000,  0.1384,  0.1507,  ..., -0.0106,  0.0624,  0.0405],
        [ 0.1384,  1.0000,  0.0033,  ..., -0.0349,  0.0558, -0.0100],
        [ 0.1507,  0.0033,  1.0000,  ...,  0.1954,  0.1487,  0.4028],
        ...,
        [-0.0106, -0.0349,  0.1954,  ...,  1.0000,  0.1168,  0.0103],
        [ 0.0624,  0.0558,  0.1487,  ...,  0.1168,  1.0000,  0.1462],
        [ 0.0405, -0.0100,  0.4028,  ...,  0.0103,  0.1462,  1.0000]],
       device='cuda:0')

In [ ]:
pairs = []
for i in range(len(cosine_similarities) - 1):
    for j in range(i + 1, len(cosine_similarities)):
        pairs.append({"index": [i, j], "score": cosine_similarities[i][j]})

# Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x["score"], reverse=True)
print(pairs)
for pair in pairs[0:10]:
    i, j = pair["index"]
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        news_cleaned.loc[4, "ID"], news_cleaned['ID'], pair["score"]
    ))

[]


In [ ]:
del query

In [ ]:
del model